In [0]:
import tensorflow as tf
import numpy as np
import h5py
import math
import time
from random import shuffle
import matplotlib.pyplot as plt
from tensorflow.contrib.layers import fully_connected, l2_regularizer
from tensorflow.contrib.framework import arg_scope
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import metrics
%matplotlib inline
import os
from glob import glob
import cv2


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from zipfile import ZipFile 

In [0]:
Xtrain = "/content/drive/My Drive/CNN R7 Project /train.zip"

In [0]:
with ZipFile(Xtrain, 'r') as zip: 
  #zip.printdir()
  zip.extractall()
  !ls

drive  sample_data  train


In [0]:
!ls train

 Black-grass	    'Common wheat'	'Scentless Mayweed'
 Charlock	    'Fat Hen'		'Shepherds Purse'
 Cleavers	    'Loose Silky-bent'	'Small-flowered Cranesbill'
'Common Chickweed'   Maize		'Sugar beet'


In [0]:
#save the path into variable to use it easily next times.
TRAin_DATA_FOLDER= "train"

In [0]:
images = []
classes=[]
missing=0
for class_folder_name in os.listdir(TRAin_DATA_FOLDER):
    class_folder_path = os.path.join(TRAin_DATA_FOLDER, class_folder_name)
    class_label = class_folder_name
    
    for image_path in glob(os.path.join(class_folder_path, "*.png")):
        image_bgr = cv2.imread(image_path, cv2.IMREAD_COLOR)
        if image_bgr is None: # if the file contain any missig value ignore it
          missing += 1
          continue
        images.append(image_bgr)
        classes.append(class_label)

In [0]:
len(images)

4750

In [0]:
# Resizing Images
def resize_images(img):

  img = np.array(img).astype(np.uint8)
  #print(img.dtype)
  res = cv2.resize(img,(256,256), interpolation = cv2.INTER_CUBIC)
  return res

In [0]:
#save resized images into images.
images = [resize_images(img) for img in images]

In [0]:
images[0].shape

(256, 256, 3)

In [0]:

#see number of images in each lablel
images = np.array(images)
classes = np.array(classes)
print("images shape: ", images.shape)
print("classes shape: ", classes.shape)
#for key,value in images_per_class.items():
   # print("number of {0} images is  -> {1}".format(key, len(value)))
  
for label in set(classes):

  print("number of {} images is  -> {}".format(
        label, 
        len(images[classes == label])))

images shape:  (4750, 256, 256, 3)
classes shape:  (4750,)
number of Fat Hen images is  -> 475
number of Common Chickweed images is  -> 611
number of Maize images is  -> 221
number of Scentless Mayweed images is  -> 516
number of Loose Silky-bent images is  -> 654
number of Cleavers images is  -> 287
number of Shepherds Purse images is  -> 231
number of Small-flowered Cranesbill images is  -> 496
number of Common wheat images is  -> 221
number of Black-grass images is  -> 263
number of Charlock images is  -> 390
number of Sugar beet images is  -> 385


In [0]:
def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    sensitivity = 40
    lower_hsv = np.array([40 - sensitivity, 80, 40])
    upper_hsv = np.array([40 + sensitivity, 255, 255])

    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

def segment_plant(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output

def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

In [0]:
def segment(img):

  image_mask = create_mask_for_plant(img)
  image_segmented = segment_plant(img)
  image_sharpen = sharpen_image(image_segmented)
  return image_sharpen

In [0]:
for indx, image in enumerate(images):
  try:
     images[indx] = segment(image)
      
  except:
    print(indx)
    continue

In [0]:
from sklearn.preprocessing import LabelBinarizer
enc = LabelBinarizer()
y = enc.fit_transform(classes)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, y, test_size=0.2, random_state=00)

In [0]:
X_train.shape

(3800, 256, 256, 3)

In [0]:
from sklearn.model_selection import train_test_split
X_test, X_val, y_test, Y_val = train_test_split(X_test,y_test, test_size = 0.6, random_state=00)

In [0]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalMaxPooling2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.layers import Flatten

# Set the CNN model 

batch_size = None

model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu', batch_input_shape = (batch_size,256, 256, 3)))
model.add(MaxPool2D())
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters = 64, kernel_size = (1,1),padding = 'Same', activation ='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters = 64, kernel_size = (1,1),padding = 'Same', activation ='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(25, activation = "relu"))
model.add(Dense(12, activation = "softmax"))
model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 256, 256, 64)      1792      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 128, 128, 128)     73856     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 64, 64, 64)        8256      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 32, 32, 32)       

In [0]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = 'adam', loss = "categorical_crossentropy", metrics = ["accuracy"])

In [0]:
model.fit(X_train, y_train, epochs = 25, validation_data = (X_val,Y_val))

Train on 3800 samples, validate on 570 samples
Epoch 1/25
3800/3800 [==============================] - 900s 237ms/step - loss: 2.6041 - acc: 0.1234 - val_loss: 2.4605 - val_acc: 0.1298
Epoch 2/25
3800/3800 [==============================] - 894s 235ms/step - loss: 2.4497 - acc: 0.1321 - val_loss: 2.4430 - val_acc: 0.1298
Epoch 3/25
3800/3800 [==============================] - 901s 237ms/step - loss: 2.4333 - acc: 0.1511 - val_loss: 2.4914 - val_acc: 0.1789
Epoch 4/25
3800/3800 [==============================] - 897s 236ms/step - loss: 2.4167 - acc: 0.1637 - val_loss: 2.5086 - val_acc: 0.2070
Epoch 5/25
3800/3800 [==============================] - 905s 238ms/step - loss: 2.3381 - acc: 0.2045 - val_loss: 2.2500 - val_acc: 0.2614
Epoch 6/25
3800/3800 [==============================] - 907s 239ms/step - loss: 2.1381 - acc: 0.2763 - val_loss: 2.0871 - val_acc: 0.2772
Epoch 7/25
3800/3800 [==============================] - 887s 234ms/step - loss: 1.9845 - acc: 0.3234 - val_loss: 2.0331 - val